<a href="https://colab.research.google.com/github/lkwate/neural-question-generation/blob/master/Question_generation_Transformer_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#install package
!pip install torch torchvision
!pip install pytorch-transformers
!pip install pytorch-pretrained-bert

In [34]:
#Download the SQUAD train and dev dataset
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-02-10 15:15:36--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json.1’

train-v2.0.json.1   100%[===================>]  40.17M   178MB/s    in 0.2s    

2020-02-10 15:15:37 (178 MB/s) - ‘train-v2.0.json.1’ saved [42123633/42123633]

--2020-02-10 15:15:39--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json.1’

dev-v2.0.json.1

In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import json
import random
from tqdm import tqdm
from pytorch_transformers import BertTokenizer
#from transformers import BertTokenizer
from nltk.tokenize import sent_tokenize # to tokenize paragraph in sentence
import nltk # to compute BLEU score of outputs question

## import about transformer model
import torch
import torch.nn as nn 
import torch.nn.functional as F
import math 
from torch.nn.init import xavier_uniform_
from torch import optim
from pytorch_pretrained_bert import BertModel

from torch.nn import Transformer
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## definition of constant

In [0]:
constant = {
    'd_model' : 512,  ## dimension of simple model's features
    'nhead' : 8,   ## number of head in multihead attention
    'max_question_length' : 30,  ## max length of question, 
    'number_layer' : 6,  ## depth of stack of layers
    'learning_rate' : 5e-5,  ## learning rate of optimizer
    'vocab_size' : 30522,  ## vocabulary size
    'dropout' :  0.1,  ## dropout hyperparameter for regularization
    'd_emb' : 768, ## dimension of word embeddings provide by bertModel-base-uncased
    'start_answer_token' : 1,  ## token follow by the answer spanned in the context 
    'end_answer_token' : 2,  ## the end token of the answer spanned in the context
    'pad' : 0,  ## pad token
    'cls' : 101,  ## cls token, begin token of sequence
    'sep' : 102,  ## separate token
    'mask' : 103,  ## mask token
    'batch_size' : 4, ## batch size
    'epoch' : 2 ## number of times training will be repeat
}

# Tokenizer

In [0]:
class Tokenizer():
    
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    def tokenize(self, input):
        return self.tokenizer.tokenize(input)
    
    def processContextAnswer(self, context, start_answer, end_answer): 
        before_answer = self.tokenizer.encode(context[:start_answer], add_special_tokens = False)
        answer = self.tokenizer.encode(context[start_answer : end_answer], add_special_tokens = False)
        after_answer = self.tokenizer.encode(context[end_answer:], add_special_tokens = False)
        indexed_tokens = [constant['cls']] + before_answer + [constant['start_answer_token']] + answer + [constant['end_answer_token']] + after_answer + [constant['sep']]
        segments_tokens = [1] * len(indexed_tokens)
        
        return indexed_tokens, segments_tokens
    
    def processQuestion(self, question):
        question_tokens = [constant['cls']] + self.tokenizer.encode(question, add_special_tokens = False) + [constant['sep']]
        question_segments = [1] * len(question_tokens)
        
        return question_tokens, question_segments

    def decode(self, input_ids):
        output = self.tokenizer.convert_ids_to_tokens(input_ids)
        output = self.tokenizer.convert_tokens_to_string(output)
        return output

# Position encoder layer

In [0]:
class PositionEncoder(nn.Module): 
    
    def __init__(self, d_model, dropout=0.1, max_len=2500):
        super(PositionEncoder, self).__init__()
        
        self.dropout = nn.Dropout(p=dropout).cuda(device)
        pe = torch.zeros(max_len,d_model).to(device)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1).to(device)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        div_term = div_term.to(device)
        pe[:, 0::2] = torch.sin(position * div_term).to(device)
        pe[:, 1::2] = torch.cos(position * div_term).to(device)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        #shape of x : (x.size(0), d_model)
        return self.dropout(x).to(device)

# Embedding layer

In [0]:
class Embedding(nn.Module):
    
    def __init__(self, d_model = 512):
        super(Embedding, self).__init__()
        
        self.d_model = d_model
        #layer to embed tokens with BERT 
        self.bertModel = BertModel.from_pretrained('bert-base-uncased').cuda(device)
        #layer to projectize output of BERT on space of input of Transformer
        self.linear = nn.Linear(constant['d_emb'], d_model).cuda(device)
        
    def forward(self, indexed_tokens, segments_ids):
        
        #get words embedding
        encoders_layers, _ = self.bertModel(indexed_tokens, segments_ids)
        #get the output of the last hidden layers of bertModel
        encoders_layers = encoders_layers[-1].squeeze(0).to(device)
        #projectize the result on dimension of model
        encoders_layers = self.linear(encoders_layers)
        #mutiply by the square root of the d_model dimension of model
        encoders_layers = encoders_layers * torch.sqrt(torch.FloatTensor([constant['d_model']]).to(device))
        #transpose (0, 1) to change the shape to (length_sequence, batch, 512)
        return encoders_layers
        

In [0]:
class TransformerModel(nn.Module):
    
    def __init__(self, d_model = 512, vocab_size = 30522):
        super(TransformerModel, self).__init__()
        
        self.transformer = Transformer(d_model = d_model, nhead=constant['nhead'], dropout=constant['dropout'], 
                                       num_encoder_layers=constant['number_layer'], 
                                       num_decoder_layers=constant['number_layer']).cuda(device)
        #position encoder
        self.position_encoder = PositionEncoder(d_model).cuda(device)
        #embedding layer for vector representation of the combination of context and answer
        self.embedding = Embedding(d_model).cuda(device)
        # layer for the projection of the output of transformer decoder on the space which the dimension is the size of vocabulary
        self.linear = nn.Linear(d_model, vocab_size).cuda(device)
        #compute log of the probality distribution
        self.log_softmax = nn.LogSoftmax(dim = -1).cuda(device)
        
    def forward(self, indexed_tokens_context = None, segments_ids_context = None, indexed_tokens_question = None, segments_ids_question = None, memory = None, indexed_tokens = None, segments_ids = None, decode = False):
        
        if decode : 
            indexed_tokens = torch.tensor([indexed_tokens]).to(device)
            segments_ids = torch.tensor([segments_ids]).to(device)
            tgt = self.embedding(indexed_tokens, segments_ids)
            tgt = tgt.unsqueeze(1)
            tgt = self.position_encoder(tgt)
            #generate mask 
            tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.shape[0]).to(device)
            output = self.transformer.decoder(tgt, memory, tgt_mask = tgt_mask)
            output = output.squeeze(1)
            output = self.linear(output)
            output = self.log_softmax(output)

            del indexed_tokens, segments_ids
            return output
        else :
            #convert to indices and segment to tensor
            indexed_tokens_context = torch.tensor([indexed_tokens_context]).to(device)
            segments_ids_context = torch.tensor([segments_ids_context]).to(device)
            indexed_tokens_question = torch.tensor([indexed_tokens_question]).to(device)
            segments_ids_question = torch.tensor([segments_ids_question]).to(device)

            # get embedding representation of context and answer combination
            src = self.embedding(indexed_tokens_context, segments_ids_context)

            src = src.unsqueeze(1)
            # add position features
            src = self.position_encoder(src)

            #get embedding representation of question
            tgt = self.embedding(indexed_tokens_question, segments_ids_question)

            tgt = tgt.unsqueeze(1)
            # add position features
            tgt = self.position_encoder(tgt)
            #generate mask
            tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.shape[0]).to(device)
            #compute memory 
            memory = self.transformer.encoder(src)
            #output
            output = self.transformer.decoder(tgt, memory, tgt_mask = tgt_mask)
            output = output.squeeze(1)
            output = self.linear(output)
            output = self.log_softmax(output)

            del indexed_tokens_context, indexed_tokens_question, segments_ids_context, segments_ids_question
            # return memory, and log of probabilities distribution of next tokens
            return memory, output

## Building of dataset based on SQUAD 2.0

In [0]:
#dataset
class Dataset(): 
    '''
        rule of validation of data in dataset: 
            context cannot be null
            answer a question regarding the context cannot be impossible
    '''
    def __init__(self, path_to_dataset): 
        self.dataset = []
        with open(path_to_dataset) as json_file: 

            data = json.load(json_file)
            for batch_data in data['data']: 
            
                for paragraph in batch_data['paragraphs']:
                    if paragraph['context']: 
                        context = paragraph['context']
                        
                        # loop over question and answers for a given context
                        if len(paragraph['qas']) != 0:
                            for qas in paragraph['qas']: 
                                if not qas['is_impossible'] : 
                                    question = qas['question']
                                    
                                    #ignore question which length is less than 3
                                    if len(question) <= 2 :
                                        continue
                                        
                                    # loop over answers
                                    length_answer = -1
                                    start_answer = 10e4
                                    end_answer = None
                                    for ans in qas['answers']: 
                                        if ans['answer_start'] < start_answer:
                                            start_answer = ans['answer_start']
                                        if length_answer <= len(ans['text']): 
                                            end_answer = start_answer + len(ans['text'])
                                        response = context[start_answer : end_answer]
                                        
                                    if response == '':
                                        continue
                                        
                                    index = 0
                                    for sentence in sent_tokenize(context):
                                        j = sentence.find(response)
                                        if j != -1 and index <= start_answer and (index + len(sentence))>= end_answer: 
                                            self.dataset.append((sentence, question, j, j + len(response)))
                                            break
                                        index += len(sentence)
        #shuffle item in dataset
        random.shuffle(self.dataset)
        

## train

In [0]:
def trainStep(model, optimizer, criterion, tokenizer, batch):
    
    optimizer.zero_grad()
    loss = 0
    for batch_item in batch:
        context = batch_item[0]
        question = batch_item[1]
        start_answer = batch_item[2]
        end_answer = batch_item[3]
        
        question_predicted = []
        indexed_tokens_question, segments_ids_question = tokenizer.processQuestion(question)
        indexed_tokens_context, segments_ids_context = tokenizer.processContextAnswer(context, start_answer, end_answer)
        
        memory, output = model(indexed_tokens_context=indexed_tokens_context, segments_ids_context=segments_ids_context, indexed_tokens_question=[indexed_tokens_question[0]], segments_ids_question=[segments_ids_question[0]])
        
        for qi in range(1, len(indexed_tokens_question) - 1):
            target_index = torch.argmax(output[-1, :]).item()
            question_predicted.append(target_index)
            loss += criterion(output[-1, :].unsqueeze(0), torch.LongTensor([indexed_tokens_question[qi]]).to(device))
            output = model(memory = memory, indexed_tokens=indexed_tokens_question[: qi + 1], segments_ids=segments_ids_question[: qi + 1], decode = True)
        #last tokens
        loss += criterion(output[-1, :].unsqueeze(0), torch.LongTensor([indexed_tokens_question[-1]]).to(device))
        target_index = torch.argmax(output[-1, :]).item()
        question_predicted.append(target_index)
        
    #compute gradient of loss function
    loss.backward()
    
    #update parameter
    optimizer.step()
    
    del indexed_tokens_context, indexed_tokens_question, output
    
    return loss, question_predicted, question

## train Iter

In [0]:
def trainTBertIter(dataset, model, optimizer, criterion, tokenizer, num_epoch = 10, period_display = 17): 
    """
        loop over dataset and train on each sample.
    """
    iter = 1
    plot_losses = []
    plot_losses_epoch = []
    plot_total_loss = 0
    plot_total_loss_epoch = 0
    
    #number of iteration in one batch
    batch_size = constant['batch_size']
    niter = len(dataset) // batch_size
    remain = len(dataset) == niter * batch_size
    
    for ep in tqdm(range(constant['epoch'])): 
        for j in tqdm(range(niter)):
            batch = dataset[j * batch_size : (j + 1) * batch_size]
            loss, last_predicted, last_truth = trainStep(model, optimizer, criterion, tokenizer, batch)
            plot_total_loss += loss.item()
            plot_total_loss_epoch += loss.item()
            
            if j % period_display == 1:
                avg_loss = plot_total_loss / (2 * batch_size)
                plot_losses.append(avg_loss)
                last_predicted = tokenizer.decode(last_predicted)
                print("loss = %.7f " %(avg_loss))
                print("predicted : ", last_predicted)
                print("truth : ", last_truth)
                plot_total_loss = 0
        if remain :
            batch = dataset[niter * batch_size :]
            loss, last_predicted, last_truth = trainStep(model, optimizer, criterion, tokenizer, batch)
            plot_total_loss = loss.item()
            plot_total_loss_epoch += loss.item()
            avg_loss = plot_total_loss / (len(dataset) - niter * batch_size)
            plot_losses.append(avg_loss)
            plot_total_loss = 0
            
            
        plot_losses_epoch.append(plot_total_loss_epoch)
        plot_total_loss_epoch = 0
        
        #save model
        pathCheckpoint = './checkpointModel.pth'   
        checkPointModel = {
            'model' : model, 
            'optimizer' : optimizer, 
            'model_state_dict' : model.state_dict(), 
            'optimizer_state_dict' : optimizer.state_dict(),
            'epoch' : ep, 
            'plot_losses' : plot_losses, 
            'plot_losses_epoch' : plot_losses_epoch
        }
        torch.save(checkPointModel, pathCheckpoint)
    return plot_losses, 

## evaluate

In [0]:
def evaluate(model, context, start_answer, end_answer, tokenizer, max_length = constant['max_question_length']):
    question_predicted = []
    with torch.no_grad():
        indexed_tokens_context, segments_ids_context = tokenizer.processContextAnswer(context, start_answer, end_answer)
        indexed_tokens_question = [constant['cls']]
        segments_ids_question  = [1]
        memory, output = model(indexed_tokens_context=indexed_tokens_context, segments_ids_context=segments_ids_context, indexed_tokens_question=indexed_tokes_question, segments_ids_question=segments_ids_question)
        for i in range(max_length):
            target_index = torch.argmax(output[-1, :]).item()
            question_predicted.append(target_index)
            if target_index == 102:
                break
            indexed_tokens_question.append(target_index)
            segments_ids_question.append(1)
            output = model(memory = memory, indexed_tokens=indexed_tokens_question, segments_ids=segments_ids_question, decode = True)

        question_predicted = tokenizer.tokenizer.decode(question_predicted)
    return question_predicted

In [0]:
model = TransformerModel()
#dataParallel
model = nn.DataParallel(model)
model = model.to(device)

In [0]:
adamOptimizer = optim.Adam(model.parameters(), lr=constant['learning_rate'])

In [0]:
criterion = nn.CrossEntropyLoss()

In [0]:
tokenizer = Tokenizer()

In [0]:
# load dataset 
dataset = Dataset('./train-v2.0.json')

In [0]:
#trainIter
trainTBertIter(dataset.dataset, model, adamOptimizer, criterion, tokenizer)





  0%|          | 0/2 [00:00<?, ?it/s]




  0%|          | 0/19417 [00:00<?, ?it/s]




  0%|          | 1/19417 [00:03<17:35:22,  3.26s/it]




  0%|          | 2/19417 [00:06<17:27:14,  3.24s/it]

loss = 114.5551643 
predicted :  [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]
truth :  Where can you obtain information about the Sunshine Act?







  0%|          | 3/19417 [00:10<18:32:03,  3.44s/it]




  0%|          | 4/19417 [00:13<18:26:21,  3.42s/it]




  0%|          | 5/19417 [00:17<18:27:31,  3.42s/it]




  0%|          | 6/19417 [00:20<18:56:30,  3.51s/it]




  0%|          | 7/19417 [00:24<19:28:38,  3.61s/it]




  0%|          | 8/19417 [00:28<20:03:59,  3.72s/it]




  0%|          | 9/19417 [00:31<18:58:34,  3.52s/it]




  0%|          | 10/19417 [00:34<18:04:27,  3.35s/it]




  0%|          | 11/19417 [00:38<19:20:20,  3.59s/it]




  0%|          | 12/19417 [00:41<18:08:17,  3.36s/it]




  0%|          | 13/19417 [00:44<17:52:01,  3.31s/it]




  0%|          | 14/19417 [00:47<17:27:06,  3.24s/it]




  0%|          | 15/19417 [00:51<17:16:24,  3.21s/it]




  0%|          | 16/19417 [00:54<17:16:11,  3.20s/it]




  0%|          | 17/19417 [00:58<19:12:39,  3.56s/it]




  0%|          | 18/19417 [01:01<18:04:53,  3.36s/it]




  0%|          | 19/19417 [01:05<19:40:28,  3.65s/it]

loss = 989.1640053 
predicted :  [SEP] [SEP] [SEP] [SEP] ? [SEP] [SEP] ? [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] ? ? [SEP] [SEP] [SEP] ?
truth :  Along with the PS Slim's value, quietness, and lower power usage, what superficial quality did PC World praise?







  0%|          | 20/19417 [01:09<19:49:01,  3.68s/it]




  0%|          | 21/19417 [01:13<19:50:58,  3.68s/it]




  0%|          | 22/19417 [01:16<19:25:25,  3.61s/it]




  0%|          | 23/19417 [01:20<19:12:34,  3.57s/it]




  0%|          | 24/19417 [01:24<19:54:21,  3.70s/it]




  0%|          | 25/19417 [01:28<21:07:12,  3.92s/it]




  0%|          | 26/19417 [01:33<22:10:21,  4.12s/it]




  0%|          | 27/19417 [01:36<21:28:24,  3.99s/it]




  0%|          | 28/19417 [01:40<20:04:24,  3.73s/it]




  0%|          | 29/19417 [01:44<20:35:52,  3.82s/it]




  0%|          | 30/19417 [01:46<19:00:19,  3.53s/it]




  0%|          | 31/19417 [01:50<19:18:36,  3.59s/it]




  0%|          | 32/19417 [01:54<19:04:02,  3.54s/it]




  0%|          | 33/19417 [01:58<19:49:34,  3.68s/it]




  0%|          | 34/19417 [02:01<19:50:37,  3.69s/it]




  0%|          | 35/19417 [02:04<18:30:00,  3.44s/it]




  0%|          | 36/19417 [02:07<17:25:39,  3.24s/it]

...






  3%|▎         | 666/19417 [39:31<18:40:34,  3.59s/it]




  3%|▎         | 667/19417 [39:35<19:28:55,  3.74s/it]




  3%|▎         | 668/19417 [39:37<17:55:37,  3.44s/it]




  3%|▎         | 669/19417 [39:41<18:48:47,  3.61s/it]




  3%|▎         | 670/19417 [39:44<17:37:11,  3.38s/it]




  3%|▎         | 671/19417 [39:48<18:00:42,  3.46s/it]




  3%|▎         | 672/19417 [39:52<19:50:41,  3.81s/it]




  3%|▎         | 673/19417 [39:58<21:49:52,  4.19s/it]




  3%|▎         | 674/19417 [40:02<21:27:23,  4.12s/it]




  3%|▎         | 675/19417 [40:04<19:37:51,  3.77s/it]




  3%|▎         | 676/19417 [40:07<17:47:04,  3.42s/it]




  3%|▎         | 677/19417 [40:10<17:06:12,  3.29s/it]




  3%|▎         | 678/19417 [40:14<18:10:10,  3.49s/it]




  3%|▎         | 679/19417 [40:17<17:46:29,  3.41s/it]




  4%|▎         | 680/19417 [40:21<18:11:21,  3.49s/it]




  4%|▎         | 681/19417 [40:25<19:10:19,  3.68s/it]




  4%|▎         | 682/19417 [40:29<19:29:36,  3.75s/

loss = 651.9264278 
predicted :  what did did the the s of the ' of ? the the the the ? ? the s ? the ? [SEP]
truth :  Who theorized that the types of media people use to communicate will offer different possibilities for the shape of society?







  4%|▎         | 683/19417 [40:33<19:55:58,  3.83s/it]




  4%|▎         | 684/19417 [40:36<18:53:37,  3.63s/it]




  4%|▎         | 685/19417 [40:40<19:13:59,  3.70s/it]




  4%|▎         | 686/19417 [40:44<19:46:17,  3.80s/it]




  4%|▎         | 687/19417 [40:48<20:10:24,  3.88s/it]




  4%|▎         | 688/19417 [40:54<22:47:51,  4.38s/it]




  4%|▎         | 689/19417 [40:58<22:37:46,  4.35s/it]




  4%|▎         | 690/19417 [41:02<22:04:41,  4.24s/it]




  4%|▎         | 691/19417 [41:06<21:20:02,  4.10s/it]




  4%|▎         | 692/19417 [41:09<19:43:26,  3.79s/it]




  4%|▎         | 693/19417 [41:12<19:21:40,  3.72s/it]




  4%|▎         | 694/19417 [41:15<18:27:52,  3.55s/it]




  4%|▎         | 695/19417 [41:18<17:29:23,  3.36s/it]




  4%|▎         | 696/19417 [41:22<17:52:57,  3.44s/it]




  4%|▎         | 697/19417 [41:25<17:26:53,  3.36s/it]




  4%|▎         | 698/19417 [41:30<19:44:51,  3.80s/it]




  4%|▎         | 699/19417 [41:33<18:52:25,  3.63s/

loss = 682.5982704 
predicted :  what did the ? the the ? ? the ? [SEP]
truth :  Who was responsible for supply humanitarian aid to civilians?







  4%|▎         | 700/19417 [41:37<18:42:37,  3.60s/it]




